In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math




# Import Sheet as csv 

In [2]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

# Format sheet 

In [3]:
ps = ps[["Date", "User", "File", "Post patch?", "Notes", "Conc.", ">400", "Ratio", "SM_QC_PF", "Bad dates"]]
ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["Conc."] = pd.to_numeric(ps["Conc."].str.replace(' ',''), errors ='force')
ps[">400"] = pd.to_numeric(ps[">400"].str.replace(' ',''), errors ='force')
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["Bad dates"] != "x"]
ps.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
3,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.00,0.81,pass,NaN
16,2016-04-06,P5,Ndnf - brl_160406_03,Outside-Out,wiped with kim wipe without water before loadi...,100.61,37.65,NaN,fail,NaN
17,2016-04-06,P5,Ndnf - brl_160406_04,Nucleated,NaN,209.76,167.00,0.69,pass,NaN
18,2016-04-06,P5,Ndnf - brl_160406_05,Partial-Nucleus,cell swelled,125.58,92.00,0.61,pass,NaN
20,2016-04-06,P5,Ndnf - brl_160406_07,partial Nucleated,NaN,135.18,128.00,0.87,pass,NaN


In [4]:
ps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2693 entries, 3 to 6064
Data columns (total 10 columns):
Date           2693 non-null datetime64[ns]
User           2688 non-null category
File           2692 non-null object
Post patch?    2693 non-null category
Notes          1610 non-null object
Conc.          2605 non-null float64
>400           2654 non-null float64
Ratio          2603 non-null object
SM_QC_PF       2693 non-null category
Bad dates      123 non-null object
dtypes: category(3), datetime64[ns](1), float64(2), object(4)
memory usage: 176.4+ KB


# Percentage of nucleated patches passing SM_QC 

In [5]:
mask = ps["Post patch?"] == "Nucleated"
mask
nuc = ps[mask]
nuc
nuc_count = nuc["Post patch?"].count()
nuc_count

693

In [6]:
ps["Post patch?"].value_counts()

Outside-Out                       876
No-Seal                           739
Nucleated                         693
Partial-Nucleus                   336
Entire-Cell                        29
no-seal                             5
Entire-Cell?                        3
partial Nucleated                   2
no -seal                            1
n                                   1
missing                             1
leaky Outside-Out                   1
outside-out                         1
Outside-Out?                        1
Nucleated patch                     1
Nucleated (maybe Entire-Cell?)      1
nucelated                           1
nucleus                             1
Name: Post patch?, dtype: int64

In [7]:
pas = ps["SM_QC_PF"] == "pass"
pas.head()

3      True
16    False
17     True
18     True
20     True
Name: SM_QC_PF, dtype: bool

In [8]:
np = ps[mask & pas]
np.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
17,2016-04-06,P5,Ndnf - brl_160406_04,Nucleated,NaN,209.76,167.0,0.69,pass,NaN
69,2016-04-12,P5,Rorb - brl_160412_01,Nucleated,"couldnt see flourescence - cell too deep, but ...",195.16,183.0,0.87,pass,no slide in LIMS
109,2016-04-18,P5,Chat - brl_160418_02,Nucleated,"coudlnt get gigaseal, put still saw flur in pi...",135.75,116.0,0.85,pass,NaN
112,2016-04-18,P5,Chat - brl_160418_05,Nucleated,lost seal at 3 mins but still able to pull ou...,158.45,113.0,0.62,pass,NaN
125,2016-04-19,P1,Chat - kjh_160419_03,Nucleated,saw fluorescence in pipette,423.02,203.0,0.48,pass,NaN


In [9]:
nuc_pass = np["Post patch?"].count()
nuc_pass
#np["Post patch?"].count()

564

In [10]:
float(nuc_pass)/nuc_count

0.8138528138528138

# Bubbles Analysis 

In [11]:
bubbles = ps.dropna(subset = ["Notes"])
bubbles.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
3,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.00,0.81,pass,NaN
16,2016-04-06,P5,Ndnf - brl_160406_03,Outside-Out,wiped with kim wipe without water before loadi...,100.61,37.65,NaN,fail,NaN
18,2016-04-06,P5,Ndnf - brl_160406_05,Partial-Nucleus,cell swelled,125.58,92.00,0.61,pass,NaN
29,2016-04-07,P1,Ndnf - kjh_160407_03,Nucleated patch,"cell swelled, no fluorescence visible in pipette",114.09,96.00,0.73,pass,NaN
30,2016-04-07,P1,Ndnf - kjh_160407_04,Partial-Nucleus,"cell swelled a lot, no fluorescence visible in...",129.23,111.00,0.79,pass,NaN


In [12]:
def make_mask(v,x):
    mask = v["Notes"].str.contains(x)
    df = v[mask]
    return df

def pass_only(y):
    mask = y["SM_QC_PF"] == "pass"
    df2 = y[mask]
    return df2

#Should also define a function to calculate pass percentages

In [16]:
def get_count(x):    
    count = x[">400"].count()
    return count

def get_sqrtct(x):
    sqrt_ct = math.sqrt(get_count(x))
    return sqrt_ct

def calc_mean(x):
    mean = x[">400"].mean()
    return mean

def calc_std(x):
    std_dev = x[">400"].std()
    return std_dev

def calc_stderr(x):
    std_err = calc_std(x) / get_sqrtct(x)
    return std_err

def calc_pp(passing,total):
    pp = float(get_count(passing)) / get_count(total)
    return pp

# No Bubbles Stats - All Samples

In [17]:
nb = make_mask(bubbles,"No Bubbles")

nb_mean = calc_mean(nb)
print "mean", nb_mean

nb_stderr = calc_stderr(nb)
print "standard error of the mean", nb_stderr

mean 1204.97952756
standard error of the mean 75.0964256974


# No Bubbles Stats - SMQC Pass Only 

In [18]:
nb_p = pass_only(make_mask(bubbles,"No Bubbles"))

nb_mean = calc_mean(nb_p)
print "mean =", nb_mean

nb_stderr = calc_stderr(nb_p)
print "standard error of the mean =", nb_stderr

nb_pp = calc_pp(nb_p,nb)
print "pass percentage =", nb_pp

mean = 1422.10582524
standard error of the mean = 77.1113916828
pass percentage = 0.811023622047


# Small Bubbles Stats - All Samples 

In [20]:
sb = make_mask(bubbles,"Small Bubbles")

sb_mean = calc_mean(sb)
print "mean", sb_mean

sb_stderr = calc_stderr(sb)
print "standard error of the mean", sb_stderr

mean 1289.76590909
standard error of the mean 73.8746475655


# Small Bubbles Stats - SMQC Pass Only 

In [21]:
sb_p = pass_only(make_mask(bubbles,"Small Bubbles"))

sb_mean = calc_mean(sb_p)
print "mean", sb_mean

sb_stderr = calc_stderr(sb_p)
print "standard error of the mean", sb_stderr

sb_pp = calc_pp(sb_p,sb)
print "pass percentage =", sb_pp

mean 1634.91622642
standard error of the mean 88.1827531095
pass percentage = 0.752840909091


# Medium bubbles Stats - All Samples 

In [23]:
mb = make_mask(bubbles,"Medium Bubbles")

mb_mean = calc_mean(mb)
print "mean =", mb_mean

mb_stderr = calc_stderr(mb)
print "standard error of the mean =", mb_stderr

mean = 1269.43797468
standard error of the mean = 102.163034743


# Medium Bubbles Stats - SMQC Pass Only

In [27]:
mb_p = pass_only(make_mask(bubbles, "Medium Bubbles"))

mb_mean = calc_mean(mb_p)
print "mean =", mb_mean

mb_stderr = calc_stderr(mb_p)
print "standard error of the mean =", calc_stderr(mb_p)

mb_pp = calc_pp(mb_p,mb)
print "pass percentage =", mb_pp

print get_count(mb)
print get_count(mb_p)

mean = 1480.50909091
standard error of the mean = 103.721800082
pass percentage = 0.835443037975
79
66


# Large Bubbles Stats - All Samples 

In [28]:
lb = make_mask(bubbles,"Large Bubbles")

lb_mean = calc_mean(lb)
print "mean =", lb_mean

lb_stderr = calc_stderr(lb)
print "standard error of the mean", lb_stderr

mean = 1036.77
standard error of the mean 161.663318094


# Large Bubbles Stats - SMQC Pass Only 

In [30]:
lb_p = pass_only(make_mask(bubbles,"Large Bubbles"))

lb_mean = calc_mean(lb_p)
print "mean =", lb_mean

lb_stderr = calc_stderr(lb_p)
print "standard error of the mean =", lb_stderr

lb_pp = calc_pp(lb_p,lb)
print "pass percentage =", lb_pp

mean = 1399.33809524
standard error of the mean = 177.726086259
pass percentage = 0.7


# Solution up Pipette - All Samples 

In [31]:
sp = make_mask(bubbles,"Solution in Pipette Shank")

sp_mean = calc_mean(sp)
print "mean =", sp_mean

sp_stderr = calc_stderr(sp)
print "standard error of the mean =", sp_stderr

mean = 574.08125
standard error of the mean = 129.675736364


# Solution up Pipette - SMQC Pass Only 

In [34]:
sp_p = pass_only(make_mask(bubbles, "Solution in Pipette Shank"))

sp_mean = calc_mean(sp_p)
print "mean =", sp_mean

sp_stderr = calc_stderr(sp_p)
print "standard error of the mean =", sp_stderr

sp_pp = calc_pp(sp_p,sp)
print "pass percentage =", sp_pp

print get_count(sp)
print get_count(sp_p)

mean = 915.6625
standard error of the mean = 186.951539689
pass percentage = 0.5
16
8
